## Activity 5 - Travelling Salaesman Problem using GA
by Francis John N. Magallanes and John Matthew Vong

This jupyter notebook will follow the implementation in this [website](https://towardsdatascience.com/introduction-to-genetic-algorithms-including-example-code-e396e98d8bf3), geeks for geeks [website](geeksforgeeks.org/traveling-salesman-problem-using-genetic-algorithm), and this [youtube video](https://www.youtube.com/watch?v=nhT56blfRpE). 

In this case, there will 5 functions to be implemented that will do the following:

1. Initialization of the Population
2. Fitness function
3. Selection
4. Crossover
5. Mutation


As for the algorithmic implementation, it will follow the geeks for geeks [website](geeksforgeeks.org/traveling-salesman-problem-using-genetic-algorithm) and the tutorialspoint [website](https://www.tutorialspoint.com/genetic_algorithms/genetic_algorithms_fundamentals.htm). Below shows the algorithm to be used in this implementation:

1. Initialize the population randomly.
2. Determine the fitness value of the initial chromosome.
3. Until termination criteria is achieved:
    1. Select parents and perform crossover.
    2. Perform mutation.
    3. Calculation of the fitness function
    4. Survival Selection

In [ ]:
#libraries used for this genetic algorithm
import random

### Travelling Salesman Problem

The problem states that the salesman should visit every node once and return to the starting node

Graph of our Problem:

![graph of the travelling salesman problem](graph.png)

In [ ]:
#Representation of the graph above with the corresponding cost
#it will use "adjacency list" for the representation
#note this will be the basis for the fitness function

adj_list = {
    #for the node A and its connections with the connection cost
    "A" : {"B" : 3 , "C" : 6, "D" : 4, "E" : 9, "G" : 12},

    #for the node B and its connections with the connection cost
    "B" : {"A" : 3, "C": 5, "G": 6, "F" : 6},

    #for the node C and its connections with the connection cost
    "C" : {"A" : 6, "B" : 5, "D" : 2, "E" : 7, "F" : 4},

    #for the node D and its connections with the connection cost
    "D" : {"A" : 4, "C" : 2, "E" : 8, "F" : 7},

    #for the node E and its connections with the connection cost
    "E" : {"A" : 9, "C" : 7, "D" : 8, "F" : 3, "G" : 5},

    #for the node F and its connections with the connection cost
    "F" : {"B" : 6, "C" : 4, "D" : 7, "E" : 3},

    #for the node G and its connections with the connection cost
    "G" : {"A" : 12, "B" : 6, "E" : 5}
}

### Chromosome Representation

The number of genes will be n -1 (where n is number of nodes in the graph) or 6 genes. Each chromosome represent a possible path or solution to the travelling sales man problem. Using OOP, the calculation of the fitness function and the mutation of the chromosome will be  incorporated to the class

In [ ]:
class Chromosome_TSP():

    def __init__(self):
        #this will initialize chromosome through shuffling
        self.chromosome = ["B", "C", "D", "E", "F", "G"]
        random.shuffle(self.chromosome)
    
    #for the mutation of the chromosome
    #it will use swap mutation
    def mutation(self, mutation_rate : float = 0.1):
        if random.random() <= mutation_rate :
            i1, i2 = random.sample(range(7), 2) #randomly choose the places to be swapped
            self.chromosome[i1], self.chromosome[i2] = self.chromosome[i2],self.chromosome[i1]
    
    #for the calculation of the fitness of the instance of the chromomosome 
    def calculate_fitness(self, adj_list : dict):
        cost_path = 0  #this is for the results of the fitness
        temp = ["A"] + self.chromosome + ["A"] # this will be essential for the computation

        for i in range(len(self.chromosome)):
            #this will check whether temp [i] and temp[i+1] is a valid edge
            #if not, it will break the loop and the cost will be 100000 (or any larger values)
            if temp[i + 1] in adj_list[temp[i]].keys():
                cost_path = cost_path + adj_list[temp[i]][temp[i+1]]
            else:
                cost_path = 100000
                break
        
        return cost_path


### Initialization of the Population


In [ ]:
def init_population(population_size) -> list:
    return [Chromosome_TSP() for _ in range(population_size)]
